In [1]:
import geopandas as gpd

In [2]:
ct_nyc = gpd.read_file('../../../data/nyc/geo/ct-nyc-2020.geojson')

In [3]:
sw_nyc = gpd.read_parquet('../../../data/nyc/processed/sidewalkwidths_nyc.parquet').to_crs(ct_nyc.crs)

In [4]:
sw_nyc.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
N = 10 
# randomly sample N census tracts 
validation_tracts = ct_nyc.sample(N, random_state=777)
validation_tracts = gpd.GeoDataFrame(validation_tracts, crs=ct_nyc.crs, geometry='geometry')

In [6]:
gpd.GeoDataFrame(validation_tracts.iloc[0,:]).T.set_geometry('geometry').set_crs(validation_tracts.crs, allow_override=True).crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
# write a function that takes a census tract geometry, clips another gdf to it, and writes both to an output directory with BoroCT2020_NTAName_datetime.parquet 
def clip_and_write(tract, sidewalk_widths, output_dir):

    # sanity check: both geodataframes should be in the same CRS
    if tract.crs != sidewalk_widths.crs:
        raise ValueError(f"CRS of the two GeoDataFrames do not match, {tract.crs} != {sidewalk_widths.crs}")
    # check if the tract is empty
    if tract.is_empty.any():
        raise ValueError("The tract geometry is empty.")
    # check if the sidewalk widths are empty
    if sidewalk_widths.is_empty.any():
        raise ValueError("The sidewalk widths geometry is empty.")
    
    # make the output dir 
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # clip the sidewalk widths to the tract geometry
    clipped_sidewalks = sidewalk_widths.clip(tract.geometry)
    
    # get the BoroCT2020 and NTAName from the tract
    boro_ct = tract.BoroCT2020.values[0]
    nta_name = tract.NTAName.values[0]
    
    # get the current datetime
    from datetime import datetime
    now = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # create the output filename
    output_filename = f"{boro_ct}_{nta_name}_{now}.parquet"
    
    # write both the tract and clipped sidewalk widths to parquet files
    tract.to_parquet(f"{output_dir}/tract_{output_filename}")
    clipped_sidewalks.to_parquet(f"{output_dir}/sidewalks_{output_filename}")

In [8]:
# use clip_and_write on each tract 
validation_tracts.apply(lambda x: clip_and_write(gpd.GeoDataFrame(x).T.set_geometry('geometry').set_crs(validation_tracts.crs), sw_nyc, './tracts'), axis=1)

1054    None
661     None
1996    None
2069    None
1884    None
519     None
1138    None
98      None
648     None
1993    None
dtype: object